In [1]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(1)

In [3]:
from Server import  Server
from Clients import Client
from Train import Trainer

In [4]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=24)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)

parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--ensemble_flag', type=bool, default=True)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:2' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model12/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
args_train = parser_train.parse_args(args=[])


In [5]:
seed_everything(1)

In [6]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.37it/s]


In [7]:
server = Server(args_train,clients)

In [8]:
server.fed_train()

test performance: [0.15398186090568158, 0.15518323406066797, 0.18186280572761412, 0.1677203120995466, 0.16007199120541957, 0.17050389876614694, 0.17021395301778022]
Epoch: 0 | Loss: 0.0856
Epoch: 0 | Loss: 0.0827
Epoch: 0 | Loss: 0.0859
Epoch: 0 | Loss: 0.0919
Epoch: 0 | Loss: 0.0810
Epoch: 0 | Loss: 0.0917
Epoch: 0 | Loss: 0.0967
Federated training Epoch [1/200] Val Loss: 0.0783
test performance: [0.07958247453892885, 0.08523698648667499, 0.09913526021250307, 0.08969417051093219, 0.09241833662843868, 0.08902055344046796, 0.09390376788908489]
Epoch: 0 | Loss: 0.0951
Epoch: 0 | Loss: 0.0707
Epoch: 0 | Loss: 0.0900
Epoch: 0 | Loss: 0.0821
Epoch: 0 | Loss: 0.1112
Epoch: 0 | Loss: 0.0947
Epoch: 0 | Loss: 0.0816
Federated training Epoch [2/200] Val Loss: 0.0778
test performance: [0.07946541198618608, 0.08441940621051887, 0.09633762085784787, 0.08913191456398735, 0.09221236619537007, 0.0880411188181949, 0.09237669844639627]
Epoch: 0 | Loss: 0.0833
Epoch: 0 | Loss: 0.0750
Epoch: 0 | Loss: 0.0

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [9]:
server.central_train()

Epoch [1/200] Train Loss: 0.0884 Val Loss: 0.0779
Epoch [2/200] Train Loss: 0.0774 Val Loss: 0.0772
Epoch [3/200] Train Loss: 0.0767 Val Loss: 0.0765
Epoch [4/200] Train Loss: 0.0760 Val Loss: 0.0758
Epoch [5/200] Train Loss: 0.0753 Val Loss: 0.0748
Epoch [6/200] Train Loss: 0.0746 Val Loss: 0.0740
Epoch [7/200] Train Loss: 0.0739 Val Loss: 0.0740
Epoch [8/200] Train Loss: 0.0733 Val Loss: 0.0736
Epoch [9/200] Train Loss: 0.0726 Val Loss: 0.0721
Epoch [10/200] Train Loss: 0.0720 Val Loss: 0.0718
Epoch [11/200] Train Loss: 0.0714 Val Loss: 0.0707
Epoch [12/200] Train Loss: 0.0710 Val Loss: 0.0702
Epoch [13/200] Train Loss: 0.0707 Val Loss: 0.0699
Epoch [14/200] Train Loss: 0.0700 Val Loss: 0.0692
Epoch [15/200] Train Loss: 0.0696 Val Loss: 0.0690
Epoch [16/200] Train Loss: 0.0690 Val Loss: 0.0683
Epoch [17/200] Train Loss: 0.0687 Val Loss: 0.0687
Epoch [18/200] Train Loss: 0.0681 Val Loss: 0.0675
Epoch [19/200] Train Loss: 0.0676 Val Loss: 0.0672
Epoch [20/200] Train Loss: 0.0673 Val Lo

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [10]:
server.local_train()

Launch Local Training!
Epoch [1/200] Train Loss: 0.0922 Val Loss: 0.0780
Epoch [2/200] Train Loss: 0.0775 Val Loss: 0.0773
Epoch [3/200] Train Loss: 0.0766 Val Loss: 0.0763
Epoch [4/200] Train Loss: 0.0758 Val Loss: 0.0754
Epoch [5/200] Train Loss: 0.0749 Val Loss: 0.0746
Epoch [6/200] Train Loss: 0.0742 Val Loss: 0.0738
Epoch [7/200] Train Loss: 0.0734 Val Loss: 0.0728
Epoch [8/200] Train Loss: 0.0729 Val Loss: 0.0722
Epoch [9/200] Train Loss: 0.0721 Val Loss: 0.0717
Epoch [10/200] Train Loss: 0.0717 Val Loss: 0.0710
Epoch [11/200] Train Loss: 0.0711 Val Loss: 0.0703
Epoch [12/200] Train Loss: 0.0704 Val Loss: 0.0698
Epoch [13/200] Train Loss: 0.0698 Val Loss: 0.0702
Epoch [14/200] Train Loss: 0.0694 Val Loss: 0.0685
Epoch [15/200] Train Loss: 0.0683 Val Loss: 0.0678
Epoch [16/200] Train Loss: 0.0679 Val Loss: 0.0673
Epoch [17/200] Train Loss: 0.0670 Val Loss: 0.0667
Epoch [18/200] Train Loss: 0.0665 Val Loss: 0.0673
Epoch [19/200] Train Loss: 0.0656 Val Loss: 0.0657
Epoch [20/200] Tr

In [11]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(1)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune(fine_tune_epochs=20)
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/20] Train Loss: 0.0640 Val Loss: 0.0645
Epoch [2/20] Train Loss: 0.0636 Val Loss: 0.0642
Epoch [3/20] Train Loss: 0.0633 Val Loss: 0.0640
Epoch [4/20] Train Loss: 0.0631 Val Loss: 0.0641
Epoch [5/20] Train Loss: 0.0629 Val Loss: 0.0637
Epoch [6/20] Train Loss: 0.0628 Val Loss: 0.0636
Epoch [7/20] Train Loss: 0.0627 Val Loss: 0.0637
Epoch [8/20] Train Loss: 0.0625 Val Loss: 0.0635
Epoch [9/20] Train Loss: 0.0625 Val Loss: 0.0634
Epoch [10/20] Train Loss: 0.0624 Val Loss: 0.0633
Epoch [11/20] Train Loss: 0.0623 Val Loss: 0.0632
Epoch [12/20] Train Loss: 0.0622 Val Loss: 0.0632
Epoch [13/20] Train Loss: 0.0622 Val Loss: 0.0631
Epoch [14/20] Train Loss: 0.0621 Val Loss: 0.0630
Epoch [15/20] Train Loss: 0.0620 Val Loss: 0.0630
Epoch [16/20] Train Loss: 0.0619 Val Loss: 0.0630
Epoch [17/20] Train Loss: 0.0618 Val Loss: 0.0630
Epoch [18/20] Train Loss: 0.0618 Val Loss: 0.0627
Epoch [19/20] Train Loss: 0.0617 Val Loss: 0.0628
Epoch [20/20] Train Loss: 0.0616 Val Loss: 0.0627
1

In [25]:
'''import utils import plot_prob_result
args_temp=copy.deepcopy(args_train)
args_temp.dataset_paths='wf7'
test_data, test_loader = get_data(args_train,flag='test')
actual_y=[]
for idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):
    actual_y.append(seq_y)
actual_y = torch.cat([torch.flatten(t) for t in actual_y])'''

In [12]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss)
    fed_local_preds.append(fed_local_pred)
    fed_local_models.append(fed_local_model)
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.07414915443283238, 0.08909740545212814, 0.09162906327680366, 0.08211914724223826, 0.08759168286693014, 0.07979674568425303, 0.08564044110323876]
0.08428909143691776


In [13]:
local_losses=[]
local_preds=[]
local_models=[]
for i in range(args_train.number_clients):
    local_pred,local_loss,local_model=clients[i].local_evaluation()
    local_losses.append(local_loss)
    local_preds.append(local_pred)
    local_models.append(local_model)
print(local_losses)

[0.08041695084371796, 0.08744908469946008, 0.09325712222656975, 0.08830002137804277, 0.07983406686721599, 0.08646285887297293, 0.09836889615869276]


In [14]:
central_losses=[]
central_preds=[]
central_models=[]
for i in range(args_train.number_clients):
    central_pred,central_loss,central_model=server.central_evaluation(dataset=i)
    central_losses.append(central_loss)
    central_preds.append(central_pred)
    central_models.append(central_model)
print(central_losses)

[0.07551540075865103, 0.09200877592937179, 0.09616044051434895, 0.08540064513632288, 0.08885041996836662, 0.0840047773050323, 0.08917854520913264]


In [15]:
print(local_losses)
print(central_losses)
print(fed_local_losses)
print(local_fine_tune_losses)

[0.08041695084371796, 0.08744908469946008, 0.09325712222656975, 0.08830002137804277, 0.07983406686721599, 0.08646285887297293, 0.09836889615869276]
[0.07551540075865103, 0.09200877592937179, 0.09616044051434895, 0.08540064513632288, 0.08885041996836662, 0.0840047773050323, 0.08917854520913264]
[0.07414915443283238, 0.08909740545212814, 0.09162906327680366, 0.08211914724223826, 0.08759168286693014, 0.07979674568425303, 0.08564044110323876]
[0.07415353316711644, 0.08799371513703914, 0.08981677175384678, 0.08206316067084465, 0.08298212394424498, 0.08085841480132243, 0.08618789921476416]


In [16]:
df = pd.DataFrame({
    'local_losses': local_losses,
    'central_losses': central_losses,
    'fed_local_losses': fed_local_losses,
    'local_fine_tune_losses': local_fine_tune_losses
})
df.T.to_csv('losses.csv', index=False)

In [17]:
import pickle

# Save the server object
with open('../result/24/server_benchmark.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/24/clients_benchmark.pkl', 'wb') as f:
    pickle.dump(clients, f)

In [334]:
import pickle

# Save server and clients
with open('server.pkl', 'wb') as f:
    pickle.dump(server, f)

with open('clients.pkl', 'wb') as f:
    pickle.dump(clients, f)

# Load server and clients
with open('server.pkl', 'rb') as f:
    server = pickle.load(f)

with open('clients.pkl', 'rb') as f:
    clients = pickle.load(f)